## for embedding (dont run multiple times it will take time)


In [ ]:
## Download and unzip the embeddings if using for 1st time. Y
##You can also save and load from your specified path once you save in your folder

!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip -q glove.twitter.27B.zip

--2023-08-13 15:20:17--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2023-08-13 15:20:18--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2023-08-13 15:20:18--  https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [ap

In [ ]:
GLOVE_EMB_TRAINED = "glove_emb_crf_weights.pkl"
GLOVE_VECTORIZER_TRAINED = "glove_crf_tv_layer.pkl"
GLOVE_EMB=200

In [ ]:
import numpy as np
glove_embeddings_path='/content/glove.twitter.27B.100d.txt'
glove_embeddings_index={}
with open(glove_embeddings_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        glove_embeddings_index[word] = coefs

In [ ]:
#!pip install numpy==1.19.5
#!pip uninstall tensorflow
#!pip install tensorflow==2.2.0
#!pip install tensorflow-addons==0.10.0
!pip install tensorflow-addons
!pip install sklearn scipy
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 7.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=90dc578123448493d118efc13d58ef5e43293b8ce06b3ab72aea77ab41c0cb01
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import random
import pickle
import json
import pandas as pd
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
import os
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## part you can change

In [ ]:
#please upload the crf file before running the code
!cp /content/crf.py . # Done for colab
#!cp '/content/drive/MyDrive/hate rate/task 3/code (paper 1)/crf.py'
from crf import CRF as CRF_lib

cp: '/content/crf.py' and './crf.py' are the same file


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/hate rate/task 3/dataset/train.csv',sep='|')


N_CRF_TAGS = 3
LSTM_UNITS = 512
DENSE_UNITS = 50
LSTM_DROPOUT = 0.2
DENSE_DROPOUT = 0.2
EPOCHS = 2 #(Default 5, check!)
BATCH_SIZE = 32
SEED = 42

GLOVE_EMB = 100  # Assuming you're using GloVe embeddings
GLOVE_EMBEDDING_DIM=100

MAX_LEN=128
# maximum word length is 112 and maximum tokenized size is 96 (because tokenizer will ignore punctuations of 112->96)


In [ ]:
df.head()

,Id,sentence,span,bio
0,0,"Say it loud , say it clear , illegal #immigran...","{'start': [8, 9], 'end': [8, 9]}",O O O O O O O O B I O O O O O
1,1,"Islam is not a religion, its just an excuse fo...","{'start': [0, 10], 'end': [4, 13]}",B I I I I O O O O O B I I I O O O O
2,2,@user Care About Illegals Breaking U.S #Immigr...,"{'start': [40], 'end': [40]}",O O O O O O O O O O O O O O O O O O O O O O O ...
3,3,Behold the future DACA America - unless we Man...,"{'start': [10, 17], 'end': [14, 18]}",O O O O O O O O O O B I I I I O O B I O O O O ...
4,4,No that little tattoo of a flower on your ankl...,"{'start': [14, 20, 25, 26], 'end': [14, 21, 25...",O O O O O O O O O O O O O O B O O O O O B I O ...


In [ ]:
df.columns

Index(['Id', 'sentence', 'span', 'bio'], dtype='object')

# code starts here

In [ ]:
maxx=0
sent=''
for s in df['sentence']:
  s_len=len(s.split())
  if(s_len>maxx):
    sent=s
    maxx=s_len





In [ ]:
print(maxx)
print(sent)

112
What I do know is that I live on a small Island and it seems that a lot of people from various countries in the world are willing to desert their homelands and their families just to come here , in my mind that tells me a lot about said people and their home countries , most of all it highlights their weaknesses of how their own countries have let them down and how they feel the need to feed off somebody elses country , how should we English interpret this without sounding rude ? ? ? , locust ? ? ? , parasite ? ? ? , you tell me . 


In [ ]:
tag2idx={'O':0,'B':1,"I":2}

In [ ]:
new_bio=pd.Series()
for indx,data in df.iterrows():
  arr=data['bio'].split()
  arr=[tag2idx[x] for x in arr]
  new_bio.at[indx]=arr


<ipython-input-18-e14f3cbb4249>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_bio=pd.Series()


In [ ]:
df['new_bio']=new_bio

In [ ]:
train_df,test_df=train_test_split(df,test_size=0.2,random_state=50)

In [ ]:
#for training data
train_y = keras.preprocessing.sequence.pad_sequences(maxlen=MAX_LEN,
                                               sequences=train_df['new_bio'],
                                               padding="post",
                                               value=0)
train_y = [keras.utils.to_categorical(i, num_classes=N_CRF_TAGS) for i in train_y]
train_y=np.asarray(train_y)

# for test data
test_y = keras.preprocessing.sequence.pad_sequences(maxlen=MAX_LEN,
                                               sequences=test_df['new_bio'],
                                               padding="post",
                                               value=0)
test_y = [keras.utils.to_categorical(i, num_classes=N_CRF_TAGS) for i in test_y]
test_y=np.asarray(test_y)



In [ ]:
train_y.shape

(1936, 128, 3)

In [ ]:
GLOVE_EMBEDDING_DIM=100

In [ ]:
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Step 1: Fit the tokenizer on the sentence corpus
tokenizer = Tokenizer(filters='',oov_token='[OOV]')

In [ ]:
GLOVE_EMB=100

In [ ]:


tokenizer.fit_on_texts(train_df['sentence'].tolist())
train_sequence=tokenizer.texts_to_sequences(train_df['sentence'].values)
test_sequence=tokenizer.texts_to_sequences(test_df['sentence'].values)


num_tokens=len(tokenizer.word_index)

num_tokens=len(tokenizer.word_index)+1

GLOVE_EMBEDDING_DIM=100





In [ ]:
oov_count=0
not_oov=0

#for train data
from pandas._libs.lib import is_float_array
embedding_matrix=np.zeros((num_tokens,GLOVE_EMBEDDING_DIM))
for word,i in tokenizer.word_index.items():
  embedding_vector=glove_embeddings_index.get(word)
  if embedding_vector is not None:
     not_oov+=1
     embedding_matrix[i]=embedding_vector
  else:
        oov_count+=1
        # Words not found in embedding index will be have random embedding.
        # This includes the representation for "padding" and "OOV"
        random_num1 = np.random.rand(GLOVE_EMB)
        random_num2 = np.random.rand(GLOVE_EMB)
        embedding_vector = [
            r1 if r2 < 0.5 else -1 * r1
            for r1, r2 in zip(random_num1, random_num2)
        ]
        # print(embedding_vector)
        embedding_matrix[i] = embedding_vector
print('the number of words out of oov ',oov_count)
print('number of words in vocabulary is ',not_oov)





the number of words out of oov  1393
number of words in vocabulary is  5645


In [ ]:
maxx_tokenized=0
for i in range(len(train_sequence)):
  current=len(train_sequence[i])
  maxx_tokenized=max(maxx_tokenized,current)

In [ ]:
maxx_tokenized

112

In [ ]:
train_maxx=0
sent=''
for s in train_df['sentence']:
  s_len=len(s.split())
  if(s_len>train_maxx):
    sent=s
    train_maxx=s_len

print(sent)
print(len(sent.split()))

What I do know is that I live on a small Island and it seems that a lot of people from various countries in the world are willing to desert their homelands and their families just to come here , in my mind that tells me a lot about said people and their home countries , most of all it highlights their weaknesses of how their own countries have let them down and how they feel the need to feed off somebody elses country , how should we English interpret this without sounding rude ? ? ? , locust ? ? ? , parasite ? ? ? , you tell me . 
112


so this time the number of token is same as the number of word/punctuation/speical

In [ ]:

train_padded_sequences = pad_sequences(train_sequence,padding='post', maxlen=MAX_LEN)
test_padded_sequences = pad_sequences(test_sequence,padding='post', maxlen=MAX_LEN)


In [ ]:
train_y.shape

(1936, 128, 3)

In [ ]:
train_padded_sequences.shape

(1936, 128)

In [ ]:
MAX_LEN

128

In [ ]:
## MO
# crf = CRF_lib(N_CRF_TAGS, sparse_target=True)

# input_text = keras.Input(shape=(MAX_LEN, ), dtype=tf.float32)
# embedding = Embedding(
#     num_tokens,
#     GLOVE_EMB,
#     embeddings_initializer=keras.initializers.Constant(embedding_matrix),
#     trainable=True,
# )(input_text)
# x = keras.layers.Bidirectional(
#     keras.layers.LSTM(units=LSTM_UNITS,
#                       return_sequences=True,
#                       recurrent_dropout=LSTM_DROPOUT,
#                       dropout=LSTM_DROPOUT,
#                       name='lstm1'))(embedding)
# x_rnn = keras.layers.Bidirectional(
#     keras.layers.LSTM(units=LSTM_UNITS,
#                       return_sequences=True,
#                       recurrent_dropout=LSTM_DROPOUT,
#                       dropout=LSTM_DROPOUT))(embedding)
# x = keras.layers.add([x, x_rnn])  # residual connection to the first biLSTM
# base_model = keras.layers.TimeDistributed(
#     keras.layers.Dense(DENSE_UNITS, activation="relu"))(x)
# base_model = keras.layers.Dense(N_CRF_TAGS)(base_model)
# out = crf(base_model)
# model = keras.models.Model(input_text, out)
# model.compile('adam', loss=crf.loss, metrics=[crf.accuracy])
# model.summary()

In [ ]:
from keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, TimeDistributed, Layer
from keras.models import Model
class SelfAttention(Layer):
    def __init__(self, attention_units, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.attention_units = attention_units

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], self.attention_units),
                                 initializer="uniform", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(self.attention_units,),
                                 initializer="uniform", trainable=True)
        super(SelfAttention, self).build(input_shape)

    def call(self, x):
        e = tf.nn.tanh(tf.matmul(x, self.W) + self.b)
        a = tf.nn.softmax(tf.matmul(e, tf.transpose(e, perm=[0, 2, 1])))
        output = tf.matmul(a, x)
        return output

In [ ]:
crf = CRF_lib(N_CRF_TAGS, sparse_target=True)

input_text = Input(shape=(MAX_LEN, ), dtype=tf.float32)
embedding = Embedding(
    num_tokens,
    GLOVE_EMB,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=True,
)(input_text)

# Add the SelfAttention layer
attention = SelfAttention(attention_units=64)(embedding)

x = Bidirectional(
    LSTM(units=LSTM_UNITS,
         return_sequences=True,
         recurrent_dropout=LSTM_DROPOUT,
         dropout=LSTM_DROPOUT,
         name='lstm1'))(attention)

x_rnn = Bidirectional(
    LSTM(units=LSTM_UNITS,
         return_sequences=True,
         recurrent_dropout=LSTM_DROPOUT,
         dropout=LSTM_DROPOUT))(attention)

x = keras.layers.add([x, x_rnn])  # residual connection to the first biLSTM
base_model = TimeDistributed(
    Dense(DENSE_UNITS, activation="relu"))(x)
base_model = Dense(N_CRF_TAGS)(base_model)
out = crf(base_model)
model = Model(input_text, out)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 128, 100)     703900      ['input_2[0][0]']                
                                                                                                  
 self_attention_1 (SelfAttentio  (None, 128, 100)    6464        ['embedding_1[0][0]']            
 n)                                                                                               
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 128, 1024)   2510848     ['self_attention_1[0][0]'] 

In [ ]:

from tensorflow.keras.optimizers import AdamW
custom_learning_rate=0.0001
# Create an optimizer with the custom learning rate
optimizer = AdamW(learning_rate=custom_learning_rate)
EPOCHS=10
# Now you can use this optimizer while compiling your model
model.compile(optimizer=optimizer, loss=crf.loss, metrics=[crf.accuracy])
# model.compile('adam', loss=crf.loss, metrics=[crf.accuracy])
model.fit(train_padded_sequences,
          train_y,
          batch_size=BATCH_SIZE,
          validation_data=(test_padded_sequences,test_y),
          epochs=EPOCHS)

#y_pred = model.predict(X_test)



Epoch 1/10
61/61 [==============================] - 218s 3s/step - loss: 8.3649 - viterbi_accuracy: 0.9763 - val_loss: 44.4184 - val_viterbi_accuracy: 0.9712
Epoch 2/10
61/61 [==============================] - 197s 3s/step - loss: 8.0448 - viterbi_accuracy: 0.9772 - val_loss: 44.0554 - val_viterbi_accuracy: 0.9716
Epoch 3/10
61/61 [==============================] - 190s 3s/step - loss: 7.8560 - viterbi_accuracy: 0.9775 - val_loss: 43.5074 - val_viterbi_accuracy: 0.9717
Epoch 4/10
61/61 [==============================] - 190s 3s/step - loss: 7.5608 - viterbi_accuracy: 0.9781 - val_loss: 42.9736 - val_viterbi_accuracy: 0.9712
Epoch 5/10
61/61 [==============================] - 194s 3s/step - loss: 7.3881 - viterbi_accuracy: 0.9786 - val_loss: 42.6440 - val_viterbi_accuracy: 0.9713
Epoch 6/10
61/61 [==============================] - 190s 3s/step - loss: 7.2364 - viterbi_accuracy: 0.9791 - val_loss: 42.4062 - val_viterbi_accuracy: 0.9721
Epoch 7/10
61/61 [==============================] - 

In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

In [ ]:
model.evaluate(test_padded_sequences,test_y)

16/16 [==============================] - 6s 386ms/step - loss: 40.6980 - viterbi_accuracy: 0.9730


[40.69804763793945, 0.9730437994003296]

In [ ]:
# Get the model's predictions on the test data
predictions = model.predict(test_padded_sequences)

16/16 [==============================] - 6s 266ms/step


In [ ]:
predictions.shape

(485, 128, 3)

In [ ]:
predictions

array([[[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       ...,

       [[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]], dtype=float32)

In [ ]:
num=111

In [ ]:
for indx,x in enumerate(test_y[num]):
  if (x[1]==1):
    print(indx)

0


In [ ]:

for indx,x in enumerate(predictions[num]):
  if (x[1]==1):
    print(indx)

0


In [ ]:
predictions.shape
predicted_classes = tf.argmax(predictions, axis=2)

In [ ]:
predicted_classes[232]

<tf.Tensor: shape=(128,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>

In [ ]:
test_y.shape

(485, 128, 3)

In [ ]:
test_y.shape
true_classes = tf.argmax(test_y, axis=2)

In [ ]:
type(predicted_classes)
a=predicted_classes.numpy()

In [ ]:
type(true_classes)
b=true_classes.numpy()

In [ ]:
if(a[111]==b[111]).all():
  print('yes')

yes


In [ ]:
correct_count=0
total_number=0
for x,y in zip(a,b):
  for p,q in zip(x,y):
    total_number+=1
    if(p==q):
      correct_count+=1




In [ ]:
print(correct_count)
print(total_number)

60624
62080


In [ ]:




# Calculate the accuracy manually using a loop
total_samples = len(test_y)
correct_predictions = 0
# Calculate the accuracy for each token for all samples
accuracies_per_token = np.mean(predicted_classes == true_classes, axis=0)

# Calculate the average accuracy across all tokens
average_accuracy = np.mean(accuracies_per_token)

print(average_accuracy)


0.9765463917525774


## seeing how to submit the prediction

In [ ]:
sub_df=pd.read_csv('/content/drive/MyDrive/hate rate/task 3/dataset/sample_submission.csv')

In [ ]:
sub_df

,Id,bio
0,400,O O O O O O O O O O O O O O O O O O O O O O O ...
1,1544,O O O O B I I I I I
2,193,O O B I O O O O O O O O O O O O B I I I O O O ...
3,897,O O B O O O O O O O O O B
4,2450,O O O O O O O B O O O O O O O O O O O O
5,606,O O O O B O O O O O
6,80,B O O O O B I I I I I I I


In [ ]:
test_dff=pd.read_csv('/content/drive/MyDrive/hate rate/task 3/dataset/test.csv',sep='|')

In [ ]:
test_dff

,Id,sentence
0,0,"Has the penny , or should I say peseta , final..."
1,1,it painfully clear why mudslime throw these pe...
2,2,says the kike cunt trying to charge for premiu...
3,3,Well The Hoes might pull this off . Pardon the...
4,4,All of our tim hortons are staffed by those pe...
...,...,...
601,601,that is average height for women you niggas ju...
602,602,Democrats Are Not On Americas SideThey Want Op...
603,603,This is our import for the last 30 years: musl...
604,604,Karma will get you @user u kunt #mkr


In [ ]:
maxx=0
sent=''
for s in test_dff['sentence']:
  s_len=len(s.split())
  if(s_len>maxx):
    sent=s
    maxx=s_len




In [ ]:
print(maxx)
print(sent)

64
i've felt anger when i ' ve read it on forum , but we have to do our part and speaking at loud voice this fact , because , for example , i ' m from europe and this fact is not pased on tv news or in the news paper ... knows your enemies is the first pass for vicory on them ! 


In [ ]:
test_data_sequence=tokenizer.texts_to_sequences(test_dff['sentence'].values)

In [ ]:
maxx_tokenized=0
for i in range(len(test_data_sequence)):
  current=len(test_data_sequence[i])
  maxx_tokenized=max(maxx_tokenized,current)

In [ ]:
maxx_tokenized

64

In [ ]:
test_data_padded_sequences = pad_sequences(test_data_sequence,padding='post', maxlen=MAX_LEN)

In [ ]:
test_data_padded_sequences.shape

(606, 128)

In [ ]:
# Get the model's predictions on the test data
test_data_predictions = model.predict(test_data_padded_sequences)

19/19 [==============================] - 5s 233ms/step


In [ ]:
test_data_predictions.shape

(606, 128, 3)

In [ ]:
test_predicted_classes = tf.argmax(test_data_predictions, axis=2)

In [ ]:
test_predicted_classes.shape

TensorShape([606, 128])

In [ ]:
# Function to count tokens in a sentence
def count_tokens(sentence):
    return len(sentence.split())

# Calculate actual sentence lengths
test_dff['sentence_length'] = test_dff['sentence'].apply(count_tokens)

# Trim predictions based on sentence lengths
trimmed_predictions = [pred[:length].numpy().tolist() for pred, length in zip(test_predicted_classes, test_dff['sentence_length'])]

# # Add trimmed predictions to the DataFrame
test_dff['trimmed_predictions'] = trimmed_predictions

# # Now df['trimmed_predictions'] contains predictions aligned with sentence lengths
# print(df)

In [ ]:
len(test_dff['trimmed_predictions'])

606

In [ ]:
type(trimmed_predictions[10])

list

In [ ]:
print(len(test_dff['sentence'][10].split()))

45


rough

In [ ]:
df['bio'][0]

'O O O O O O O O B I O O O O O'

In [ ]:
tag2idx = {'O': 0, 'B': 1, 'I': 2}

# Example list of indices (replace with your actual list of indices)
idx_list = [0, 1, 2, 1, 2, 2, 0]

# Convert indices back to labels using tag2idx dictionary and join them into a string
labels = [key for idx in idx_list for key, value in tag2idx.items() if value == idx]
labels_string = ' '.join(labels)

print(labels_string)


O B I B I I O


In [ ]:
print(trimmed_predictions[15])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0]


In [ ]:
for l in trimmed_predictions:
  labels = [key for idx in l for key, value in tag2idx.items() if value == idx]


In [ ]:
# Convert indices to labels for each list in the 'trimmed_predictions' column
def convert_indices_to_labels(idx_list):
    labels = [key for idx in idx_list for key, value in tag2idx.items() if value == idx]
    return ' '.join(labels)

test_dff['bio'] = test_dff['trimmed_predictions'].apply(convert_indices_to_labels)


In [ ]:
test_dff

,Id,sentence,sentence_length,trimmed_predictions,bio
0,0,"Has the penny , or should I say peseta , final...",46,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",O O O O O O O O O O O O O O O O O O O O O O O ...
1,1,it painfully clear why mudslime throw these pe...,10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",O O O O O O O O O O
2,2,says the kike cunt trying to charge for premiu...,51,"[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",O O B I O O O O O O O O O O O O O O O B O O O ...
3,3,Well The Hoes might pull this off . Pardon the...,13,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",O O B O O O O O O O O O O
4,4,All of our tim hortons are staffed by those pe...,20,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",O O O O O O O O O O O O O O O O O O O O
...,...,...,...,...,...
601,601,that is average height for women you niggas ju...,10,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",O O O O O O O B O O
602,602,Democrats Are Not On Americas SideThey Want Op...,39,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, ...",O O O O O O O O O O B I O O O O O O O O O O O ...
603,603,This is our import for the last 30 years: musl...,21,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, ...",O O O O O O O O O I I I I I O O O O O O O
604,604,Karma will get you @user u kunt #mkr,8,"[0, 0, 0, 0, 0, 0, 0, 0]",O O O O O O O O


In [ ]:
output_file=test_dff[['Id','bio']]

In [ ]:
# Set 'index_column' as the index and reset the index to False
#output_file.set_index('Id', inplace=True)
output_file.reset_index(drop=True, inplace=True)

In [ ]:
output_file.to_csv('/content/drive/MyDrive/hate rate/task 3/prediction files/second.csv',index=False)

In [ ]:
check_df=pd.read_csv("/content/drive/MyDrive/hate rate/task 3/prediction files/second.csv")

In [ ]:
check_df

,Id,bio
0,0,O O O O O O O O O O O O O O O O O O O O O O O ...
1,1,O O O O O O O O O O
2,2,O O B I O O O O O O O O O O O O O O O B O O O ...
3,3,O O B O O O O O O O O O O
4,4,O O O O O O O O O O O O O O O O O O O O
...,...,...
601,601,O O O O O O O B O O
602,602,O O O O O O O O O O B I O O O O O O O O O O O ...
603,603,O O O O O O O O O I I I I I O O O O O O O
604,604,O O O O O O O O
